In [7]:
import ee
import os
import geemap
import datetime
from ee.batch import Export, Task
from dateutil.relativedelta import *
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:4780'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:4780'
ee.Initialize()

In [4]:
# ee.Authenticate()

# 直接加载roi为福建省（也就是研究区，注意，太大的shapefile不支持这种方法直接读取，请手动上传到GEE平台）

In [9]:
roi_path = r"E:\phd_l1\微信公众号\20250421_GEE注册py安装全流程\Fujian_prov.shp"
roi = geemap.shp_to_ee(roi_path)

# hourly wind speed u and v component //2010年6月福建下载示例，可调整代码循环直接下载多年数据

In [10]:
# ======================================================
# ERA5-Land小时级风场数据导出脚本
# 
# 功能：从ERA5-Land数据集导出指定时间段的10米高度u/v风分量数据到Google Drive
#
# 使用前提：
# 1. 已安装并验证Earth Engine Python API
# ======================================================

# 用户配置区：
# 设置需要导出的起始年份和月份(1-12月)
start_year = 2010  # 修改为您需要的起始年份
start_month = 6    # 修改为您需要的起始月份(1-12),如果需要批量下载多年请直接设置为1

# 主处理循环
for dy in range(1):  # 年份偏移量循环(当前设置为仅处理起始年份)
    # 创建起始日期对象(每月1号)
    start = datetime.datetime.strptime(f'{start_year}-{start_month:02d}-01', '%Y-%m-%d') + relativedelta(years = +dy)
    
    for t in range(1):  # 月份偏移量循环(当前设置为仅处理起始月份)
        # 设置时间范围(当月1号到下个月1号)
        start_time = ee.Date(str(start + relativedelta(months = +t)).split()[0])
        end_time = ee.Date(str(start + relativedelta(months = +(t+1))).split()[0])
        
        # 处理10米高度u分量风场数据
        era5_wind_u = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
                           .select('u_component_of_wind_10m') \
                           .filter(ee.Filter.date(start_time, end_time)).toBands().clip(roi).unmask(-9999)
        
        # 设置导出文件名格式：ERA5_wind_u_年份_月份
        name = f"ERA5_wind_u_{start_year+dy}_{t+start_month}"
        folder = "ERA5_wind_hourly"  # Google Drive中的目标文件夹
        
        # 创建导出任务到Google Drive
        task = ee.batch.Export.image.toDrive(image=era5_wind_u, description=name,fileNamePrefix=name, folder=folder, maxPixels=int(1e13), scale=11132, region=roi.geometry(), fileFormat='GeoTIFF')
        task.start()
        print('正在导出 {} 到Google Drive的 {} 文件夹'.format(name, folder))

        # 处理10米高度v分量风场数据（流程同上）
        era5_wind_v = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
                           .select('v_component_of_wind_10m') \
                           .filter(ee.Filter.date(start_time, end_time)).toBands().clip(roi).unmask(-9999)
        name = f"ERA5_wind_v_{start_year+dy}_{t+start_month}"
        task = ee.batch.Export.image.toDrive(image=era5_wind_v, description=name,fileNamePrefix=name, folder=folder, maxPixels=int(1e13), scale=11132, region=roi.geometry(), fileFormat='GeoTIFF')
        task.start()
        print('正在导出 {} 到Google Drive的 {} 文件夹'.format(name, folder))

# ======================================================
# 使用说明：
# 1. 修改start_year和start_month为需要导出的年月
# 2. 调整range(1)参数可导出多个月份/年份数据
# 3. 导出文件将包含指定时间段内所有小时级数据
# 4. 默认输出到Google Drive的"ERA5_wind_hourly"文件夹
# 5. 缺失值被替换为-9999
# 6. 输出分辨率为11132米(约0.1度)
# ======================================================

正在导出 ERA5_wind_u_2010_6 到Google Drive的 ERA5_wind_hourly 文件夹
正在导出 ERA5_wind_v_2010_6 到Google Drive的 ERA5_wind_hourly 文件夹
